In [1]:
print("="*60)
print("STEP 1: DOWNLOADING GOOGLE FONTS")
print("="*60)

# Install required packages
print("\n📦 Installing required packages...")
!pip install fonttools requests pillow

import os
import glob
from pathlib import Path

# Create directories
print("\n📁 Creating directories...")
fonts_dir = Path("google_fonts_repo")
fonts_dir.mkdir(exist_ok=True)

print("\n📥 Cloning Google Fonts repository...")
print("   (This may take 2-3 minutes - it's ~500MB)")

# Clone the Google Fonts GitHub repo
!git clone --depth 1 https://github.com/google/fonts.git google_fonts_repo

print("\n✅ Google Fonts repository downloaded!")

# Analyze what we got
print("\n📊 Analyzing downloaded fonts...")

# Find all .ttf files
font_files = glob.glob("google_fonts_repo/**/*.ttf", recursive=True)
print(f"\n   Total .ttf files found: {len(font_files)}")

# Separate regular fonts from variable fonts
regular_fonts = [f for f in font_files if "[" not in f]
variable_fonts = [f for f in font_files if "[" in f]

print(f"   Regular fonts: {len(regular_fonts)}")
print(f"   Variable fonts: {len(variable_fonts)}")

# Get unique font families
font_families = set()
for font_path in regular_fonts:
    # Extract font family name from path
    parts = Path(font_path).parts
    if len(parts) >= 3:
        font_families.add(parts[-2])

print(f"   Unique font families: {len(font_families)}")

# Show some examples
print("\n📝 Sample fonts:")
for i, font_path in enumerate(regular_fonts[:10]):
    font_name = Path(font_path).name
    print(f"   • {font_name}")

print(f"\n   ... and {len(regular_fonts) - 10} more")

print("\n✅ Step 1 complete! Ready for Step 2 (generating training images)")

STEP 1: DOWNLOADING GOOGLE FONTS

📦 Installing required packages...

📁 Creating directories...

📥 Cloning Google Fonts repository...
   (This may take 2-3 minutes - it's ~500MB)
Cloning into 'google_fonts_repo'...
remote: Enumerating objects: 18985, done.
remote: Counting objects: 100% (18985/18985), done.
remote: Compressing objects: 100% (15371/15371), done.
remote: Total 18985 (delta 3909), reused 12827 (delta 3050), pack-reused 0 (from 0)
Receiving objects: 100% (18985/18985), 1.18 GiB | 14.30 MiB/s, done.
Resolving deltas: 100% (3909/3909), done.
Checking connectivity: 18985, done.
Updating files: 100% (16945/16945), done.

✅ Google Fonts repository downloaded!

📊 Analyzing downloaded fonts...

   Total .ttf files found: 3817
   Regular fonts: 3083
   Variable fonts: 734
   Unique font families: 1448

📝 Sample fonts:
   • Aclonica-Regular.ttf
   • Calligraffitti-Regular.ttf
   • CherryCreamSoda-Regular.ttf
   • Chewy-Regular.ttf
   • ComingSoon-Regular.ttf
   • Cousine-Bold.ttf
  

In [2]:
print("="*60)
print("STEP 2: GENERATING TRAINING IMAGES")
print("="*60)

from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path
import random
import json

class GoogleFontsDatasetGenerator:
    def __init__(self, fonts_dir="google_fonts_repo", output_dir="training_data"):
        self.fonts_dir = Path(fonts_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        
        # Sample texts to render
        self.sample_texts = [
            "The quick brown fox jumps over the lazy dog",
            "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
            "abcdefghijklmnopqrstuvwxyz",
            "0123456789",
            "Hello World! Design Systems 2025",
            "Typography & Layout Patterns",
            "Beautiful Design Tokens",
            "Color Spacing Grid",
        ]
        
        # Image settings
        self.img_size = (320, 320)
        self.font_sizes = [28, 36, 44, 52, 60]
        
    def get_font_files(self):
        """Get all .ttf files from Google Fonts repo"""
        font_files = list(self.fonts_dir.glob("**/*.ttf"))
        # Filter out variable fonts for now
        font_files = [f for f in font_files if "[" not in f.name]
        return font_files
    
    def generate_image(self, font_path, text, font_size):
        """Generate a single training image"""
        try:
            # Create white background
            img = Image.new('RGB', self.img_size, color='white')
            draw = ImageDraw.Draw(img)
            
            # Load font
            font = ImageFont.truetype(str(font_path), font_size)
            
            # Calculate text position (centered)
            bbox = draw.textbbox((0, 0), text, font=font)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
            
            x = (self.img_size[0] - text_width) // 2
            y = (self.img_size[1] - text_height) // 2
            
            # Draw text in black
            draw.text((x, y), text, font=font, fill='black')
            
            return img
        except Exception as e:
            # Some fonts might fail to load or render
            return None
    
    def generate_dataset(self, samples_per_font=10, max_fonts=None):
        """Generate full training dataset"""
        font_files = self.get_font_files()
        
        if max_fonts:
            font_files = font_files[:max_fonts]
        
        print(f"\n🎨 Generating dataset from {len(font_files)} fonts...")
        print(f"   Creating {samples_per_font} samples per font")
        print(f"   Total target: {len(font_files) * samples_per_font} images\n")
        
        dataset_info = []
        total_generated = 0
        failed_fonts = []
        
        for font_idx, font_path in enumerate(font_files):
            # Get font family name
            font_name = font_path.stem  # e.g., "Roboto-Regular"
            
            # Extract family name (the parent directory)
            font_family = font_path.parent.name
            
            # Create directory for this font family
            font_output_dir = self.output_dir / font_family
            font_output_dir.mkdir(exist_ok=True)
            
            # Generate multiple samples
            samples_generated = 0
            for sample_idx in range(samples_per_font):
                # Random text and size
                text = random.choice(self.sample_texts)
                font_size = random.choice(self.font_sizes)
                
                # Generate image
                img = self.generate_image(font_path, text, font_size)
                
                if img:
                    # Save image
                    img_filename = f"{font_name}_{sample_idx}.png"
                    img_path = font_output_dir / img_filename
                    img.save(img_path)
                    
                    # Record metadata
                    dataset_info.append({
                        'image_path': str(img_path),
                        'font_name': font_name,
                        'font_family': font_family,
                        'text': text,
                        'font_size': font_size
                    })
                    
                    total_generated += 1
                    samples_generated += 1
            
            if samples_generated == 0:
                failed_fonts.append(str(font_path))
            
            # Progress update every 100 fonts
            if (font_idx + 1) % 100 == 0:
                print(f"   ✓ Processed {font_idx + 1}/{len(font_files)} fonts ({total_generated} images generated)")
        
        print(f"\n✅ Dataset generation complete!")
        print(f"   Total images generated: {total_generated}")
        print(f"   Unique font families: {len(set(info['font_family'] for info in dataset_info))}")
        print(f"   Failed fonts: {len(failed_fonts)}")
        
        # Save dataset metadata
        with open(self.output_dir / "dataset_info.json", 'w') as f:
            json.dump(dataset_info, f, indent=2)
        
        if failed_fonts:
            with open(self.output_dir / "failed_fonts.txt", 'w') as f:
                f.write("\n".join(failed_fonts))
        
        return dataset_info

# Create generator
generator = GoogleFontsDatasetGenerator()

# Generate dataset - starting with 200 fonts for testing
print("🚀 Starting dataset generation...")
print("   Starting with 200 fonts to test (about 2,000 images)")
print("   This should take 3-5 minutes...\n")

dataset_info = generator.generate_dataset(
    samples_per_font=10,
    max_fonts=200  # Start with 200 fonts, can increase to all 3,083 later
)

print(f"\n📊 Dataset ready!")
print(f"   Location: training_data/")
print(f"   Ready for Step 3: Model Training")

STEP 2: GENERATING TRAINING IMAGES
🚀 Starting dataset generation...
   Starting with 200 fonts to test (about 2,000 images)
   This should take 3-5 minutes...


🎨 Generating dataset from 200 fonts...
   Creating 10 samples per font
   Total target: 2000 images

   ✓ Processed 100/200 fonts (1000 images generated)
   ✓ Processed 200/200 fonts (2000 images generated)

✅ Dataset generation complete!
   Total images generated: 2000
   Unique font families: 105
   Failed fonts: 0

📊 Dataset ready!
   Location: training_data/
   Ready for Step 3: Model Training


In [4]:
print("="*60)
print("STEP 3: TRAINING THE MODEL")
print("="*60)

# Install transformers for training
print("\n📦 Installing training packages...")
!pip install transformers datasets accelerate scikit-learn

import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import numpy as np

print("\n🔧 Preparing dataset and model...")

# Load dataset from the images we generated
print("   Loading dataset from training_data/...")
dataset = load_dataset("imagefolder", data_dir="training_data")

print(f"\n📊 Dataset loaded:")
print(f"   Total samples: {len(dataset['train'])}")
print(f"   Number of font classes: {dataset['train'].features['label'].num_classes}")

# Get label information
labels = dataset['train'].features['label'].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

print(f"\n📝 Sample font families (first 10):")
for i, label in enumerate(labels[:10]):
    print(f"   {i}. {label}")
print(f"   ... and {len(labels) - 10} more")

# Load base model (ResNet-18 - same as gaborcselle used)
print("\n🤖 Loading base model (ResNet-18)...")
model_name = "microsoft/resnet-18"
processor = AutoImageProcessor.from_pretrained(model_name)

# Create model for fine-tuning
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print("   ✓ Base model loaded")

# Preprocessing function
def preprocess(examples):
    """Preprocess images for training"""
    images = [img.convert("RGB") for img in examples['image']]
    inputs = processor(images, return_tensors='pt')
    inputs['labels'] = examples['label']
    return inputs

# Apply preprocessing
print("\n🔄 Preprocessing images...")
dataset = dataset.map(preprocess, batched=True, remove_columns=['image'])
dataset.set_format('torch', columns=['pixel_values', 'labels'])

# Split into train/validation (80/20)
print("   Splitting into train/validation sets...")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

print(f"   Training samples: {len(dataset['train'])}")
print(f"   Validation samples: {len(dataset['test'])}")

# Define metrics
def compute_metrics(eval_pred):
    """Calculate accuracy during training"""
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training configuration (optimized for quick testing)
print("="*60)
print("STEP 3: TRAINING THE MODEL (CPU MODE)")
print("="*60)

# Install transformers for training
print("\n📦 Installing training packages...")
!pip install transformers datasets accelerate scikit-learn

import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import numpy as np

# Force CPU usage to avoid CUDA errors
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

print("\n🔧 Preparing dataset and model...")
print("   Running on CPU for stability")

# Load dataset from the images we generated
print("   Loading dataset from training_data/...")
dataset = load_dataset("imagefolder", data_dir="training_data")

print(f"\n📊 Dataset loaded:")
print(f"   Total samples: {len(dataset['train'])}")
print(f"   Number of font classes: {dataset['train'].features['label'].num_classes}")

# Get label information
labels = dataset['train'].features['label'].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

print(f"\n📝 Sample font families (first 10):")
for i, label in enumerate(labels[:10]):
    print(f"   {i}. {label}")
print(f"   ... and {len(labels) - 10} more")

# Load base model (ResNet-18)
print("\n🤖 Loading base model (ResNet-18)...")
model_name = "microsoft/resnet-18"
processor = AutoImageProcessor.from_pretrained(model_name)

# Create model for fine-tuning
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print("   ✓ Base model loaded")

# Preprocessing function
def preprocess(examples):
    """Preprocess images for training"""
    images = [img.convert("RGB") for img in examples['image']]
    inputs = processor(images, return_tensors='pt')
    inputs['labels'] = examples['label']
    return inputs

# Apply preprocessing
print("\n🔄 Preprocessing images...")
dataset = dataset.map(preprocess, batched=True, remove_columns=['image'])
dataset.set_format('torch', columns=['pixel_values', 'labels'])

# Split into train/validation (80/20)
print("   Splitting into train/validation sets...")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

print(f"   Training samples: {len(dataset['train'])}")
print(f"   Validation samples: {len(dataset['test'])}")

# Define metrics
def compute_metrics(eval_pred):
    """Calculate accuracy during training"""
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training configuration - REDUCED for CPU
print("\n⚙️ Configuring training for CPU...")
training_args = TrainingArguments(
    output_dir="./google_fonts_model_checkpoint",
    num_train_epochs=10,  # Reduced from 20 for CPU
    per_device_train_batch_size=8,  # Reduced from 16 for CPU
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="none",
    no_cuda=True,  # Force CPU usage
)

print(f"\n📋 Training Configuration:")
print(f"   Device: CPU")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Estimated time: 20-30 minutes on CPU")

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
)

print("\n🚀 Starting training...")
print("   This will take 20-30 minutes on CPU")
print("   You'll see progress updates every 20 steps\n")
print("="*60)

# Train!
trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)

# Evaluate final performance
print("\n📊 Final Evaluation:")
eval_results = trainer.evaluate()
print(f"   Final Accuracy: {eval_results['eval_accuracy']:.2%}")
print(f"   Final Loss: {eval_results['eval_loss']:.4f}")

# Save the final model
print("\n💾 Saving model...")
model.save_pretrained("./tesserae_google_fonts_model")
processor.save_pretrained("./tesserae_google_fonts_model")

print("\n✅ Model saved to: ./tesserae_google_fonts_model")
print("\n🎉 Training complete! Ready for Step 4: Testing")

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
)

print("\n🚀 Starting training...")
print("   This will take 15-25 minutes depending on GPU availability")
print("   You'll see progress updates every 20 steps\n")
print("="*60)

# Train!
trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)

# Evaluate final performance
print("\n📊 Final Evaluation:")
eval_results = trainer.evaluate()
print(f"   Final Accuracy: {eval_results['eval_accuracy']:.2%}")
print(f"   Final Loss: {eval_results['eval_loss']:.4f}")

# Save the final model
print("\n💾 Saving model...")
model.save_pretrained("./tesserae_google_fonts_model")
processor.save_pretrained("./tesserae_google_fonts_model")

print("\n✅ Model saved to: ./tesserae_google_fonts_model")
print("\n🎉 Training complete! Ready for Step 4: Testing")

STEP 3: TRAINING THE MODEL

📦 Installing training packages...

🔧 Preparing dataset and model...
   Loading dataset from training_data/...


Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]


📊 Dataset loaded:
   Total samples: 2000
   Number of font classes: 105

📝 Sample font families (first 10):
   0. abeezee
   1. abel
   2. abhayalibre
   3. aboreto
   4. abrilfatface
   5. abyssinicasil
   6. aclonica
   7. acme
   8. actor
   9. adamina
   ... and 95 more

🤖 Loading base model (ResNet-18)...


Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-18 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([105]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([105, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   ✓ Base model loaded

🔄 Preprocessing images...
   Splitting into train/validation sets...
   Training samples: 1600
   Validation samples: 400
STEP 3: TRAINING THE MODEL (CPU MODE)

📦 Installing training packages...

🔧 Preparing dataset and model...
   Running on CPU for stability
   Loading dataset from training_data/...


...


✅ Training complete!

📊 Final Evaluation:


   Final Accuracy: 70.75%
   Final Loss: 1.1043

💾 Saving model...

✅ Model saved to: ./tesserae_google_fonts_model

🎉 Training complete! Ready for Step 4: Testing

🚀 Starting training...
   This will take 15-25 minutes depending on GPU availability
   You'll see progress updates every 20 steps



Epoch,Training Loss,Validation Loss,Accuracy
1,0.585300,1.078112,0.717500
2,0.381800,1.113507,0.717500
3,0.229200,0.966514,0.750000
4,0.110800,0.965190,0.757500
5,0.067000,0.871591,0.760000
6,0.103700,0.896884,0.755000
7,0.077800,0.890424,0.760000
8,0.048400,0.924720,0.767500
9,0.015200,0.853834,0.777500
10,0.047000,0.866165,0.777500



✅ Training complete!

📊 Final Evaluation:


   Final Accuracy: 77.75%
   Final Loss: 0.8538

💾 Saving model...

✅ Model saved to: ./tesserae_google_fonts_model

🎉 Training complete! Ready for Step 4: Testing


In [6]:
print("="*60)
print("STEP 4: TESTING YOUR GOOGLE FONTS MODEL")
print("="*60)

from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

print("\n🤖 Loading your trained model...")

# Load your custom Google Fonts model
processor = AutoImageProcessor.from_pretrained("./tesserae_google_fonts_model")
model = AutoModelForImageClassification.from_pretrained("./tesserae_google_fonts_model")

print("✅ Model loaded!")
print(f"   Can identify {len(model.config.id2label)} font families")

def identify_google_font(image_path):
    """Identify font using your custom Google Fonts model"""
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get top 5 predictions
    top5_prob, top5_idx = torch.topk(probabilities, 5)
    
    results = []
    for prob, idx in zip(top5_prob[0], top5_idx[0]):
        results.append({
            'font_family': model.config.id2label[idx.item()],
            'confidence': prob.item()
        })
    
    return results

# Test with your test image
print("\n🧪 Testing with test4.png...")
print("="*60)

results = identify_google_font("test4.png")

print("\n🎯 GOOGLE FONTS DETECTION RESULTS:")
for i, result in enumerate(results, 1):
    print(f"  {i}. {result['font_family']:<30} {result['confidence']:>6.1%}")

print("\n" + "="*60)
print("✅ Testing complete!")
print("\n💡 Your model is ready to use in Tesserae!")


STEP 4: TESTING YOUR GOOGLE FONTS MODEL

🤖 Loading your trained model...
✅ Model loaded!
   Can identify 105 font families

🧪 Testing with test4.png...

🎯 GOOGLE FONTS DETECTION RESULTS:
  1. rochester                       11.9%
  2. craftygirls                      7.4%
  3. homemadeapple                    7.1%
  4. kosugi                           4.8%
  5. jsmathcmr10                      4.6%

✅ Testing complete!

💡 Your model is ready to use in Tesserae!


In [7]:
print("="*60)
print("REGENERATING DATASET - FULL SCALE")
print("="*60)

from pathlib import Path
import shutil

# First, let's clean up the old training data
print("\n🗑️ Removing old training data...")
training_dir = Path("training_data")
if training_dir.exists():
    shutil.rmtree(training_dir)
    print("   ✓ Old data removed")

training_dir.mkdir(exist_ok=True)

# Recreate generator
generator = GoogleFontsDatasetGenerator()

# Check how many fonts we have available
all_fonts = generator.get_font_files()
print(f"\n📊 Available fonts: {len(all_fonts)}")

# Decision point: how many to use
print("\n⚙️ Generation options:")
print("   Option A: 500 fonts × 25 samples = 12,500 images (~15 min)")
print("   Option B: 1000 fonts × 25 samples = 25,000 images (~30 min)")
print("   Option C: ALL 3,083 fonts × 25 samples = 77,075 images (~90 min)")

print("\n📝 Recommendation: Start with Option B (1000 fonts)")
print("   This gives good coverage without taking forever\n")

# Generate with 1000 fonts
print("🚀 Starting generation with 1000 fonts, 25 samples each...")
print("   This will take approximately 30 minutes")
print("   Estimated total: 25,000 images\n")

dataset_info = generator.generate_dataset(
    samples_per_font=25,  # Up from 10
    max_fonts=1000  # Up from 200
)

print("\n✅ New dataset ready!")
print("   Ready to retrain with much more data")

REGENERATING DATASET - FULL SCALE

🗑️ Removing old training data...
   ✓ Old data removed

📊 Available fonts: 3083

⚙️ Generation options:
   Option A: 500 fonts × 25 samples = 12,500 images (~15 min)
   Option B: 1000 fonts × 25 samples = 25,000 images (~30 min)
   Option C: ALL 3,083 fonts × 25 samples = 77,075 images (~90 min)

📝 Recommendation: Start with Option B (1000 fonts)
   This gives good coverage without taking forever

🚀 Starting generation with 1000 fonts, 25 samples each...
   This will take approximately 30 minutes
   Estimated total: 25,000 images


🎨 Generating dataset from 1000 fonts...
   Creating 25 samples per font
   Total target: 25000 images

   ✓ Processed 100/1000 fonts (2500 images generated)
   ✓ Processed 200/1000 fonts (5000 images generated)
   ✓ Processed 300/1000 fonts (7500 images generated)
   ✓ Processed 400/1000 fonts (10000 images generated)
   ✓ Processed 500/1000 fonts (12500 images generated)
   ✓ Processed 600/1000 fonts (15000 images generate

In [8]:
print("="*60)
print("RETRAINING MODEL WITH EXPANDED DATASET")
print("="*60)

import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import os

# Force CPU usage
os.environ['CUDA_VISIBLE_DEVICES'] = ''

print("\n🔧 Loading expanded dataset...")
dataset = load_dataset("imagefolder", data_dir="training_data")

print(f"\n📊 New Dataset Statistics:")
print(f"   Total samples: {len(dataset['train'])}")
print(f"   Number of font classes: {dataset['train'].features['label'].num_classes}")
print(f"   That's {len(dataset['train']) / 2000:.1f}x more data than before!")

# Get label information
labels = dataset['train'].features['label'].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# Load fresh base model
print("\n🤖 Loading fresh ResNet-18 model...")
model_name = "microsoft/resnet-18"
processor = AutoImageProcessor.from_pretrained(model_name)

model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print("   ✓ Fresh model loaded")

# Preprocessing
def preprocess(examples):
    images = [img.convert("RGB") for img in examples['image']]
    inputs = processor(images, return_tensors='pt')
    inputs['labels'] = examples['label']
    return inputs

print("\n🔄 Preprocessing images...")
dataset = dataset.map(preprocess, batched=True, remove_columns=['image'])
dataset.set_format('torch', columns=['pixel_values', 'labels'])

# Split train/validation
print("   Splitting into train/validation sets...")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

print(f"   Training samples: {len(dataset['train'])}")
print(f"   Validation samples: {len(dataset['test'])}")

# Metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training configuration - increased epochs for better learning
print("\n⚙️ Configuring training...")
training_args = TrainingArguments(
    output_dir="./google_fonts_model_v2_checkpoint",
    num_train_epochs=15,  # Increased from 10
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    report_to="none",
    no_cuda=True,
)

print(f"\n📋 Training Configuration:")
print(f"   Device: CPU")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Training samples: {len(dataset['train'])}")
print(f"   Estimated time: 60-90 minutes on CPU (larger dataset)")

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
)

print("\n🚀 Starting retraining...")
print("   This will take 60-90 minutes with the larger dataset")
print("   You'll see progress updates every 50 steps")
print("   Go grab some coffee! ☕\n")
print("="*60)

# Train!
trainer.train()

print("\n" + "="*60)
print("✅ Retraining complete!")
print("="*60)

# Evaluate
print("\n📊 Final Evaluation:")
eval_results = trainer.evaluate()
print(f"   Final Accuracy: {eval_results['eval_accuracy']:.2%}")
print(f"   Final Loss: {eval_results['eval_loss']:.4f}")

# Save the improved model
print("\n💾 Saving improved model...")
model.save_pretrained("./tesserae_google_fonts_model_v2")
processor.save_pretrained("./tesserae_google_fonts_model_v2")

print("\n✅ Improved model saved to: ./tesserae_google_fonts_model_v2")
print("   This should perform MUCH better than v1!")
print("\n🎉 Ready to test the improved model!")

RETRAINING MODEL WITH EXPANDED DATASET

🔧 Loading expanded dataset...


Resolving data files:   0%|          | 0/25000 [00:00<?, ?it/s]

Computing checksums:  17%|#7        | 4320/25000 [00:05<00:23, 863.87it/s]

Generating train split: 0 examples [00:00, ? examples/s]


📊 New Dataset Statistics:
   Total samples: 25000
   Number of font classes: 466
   That's 12.5x more data than before!

🤖 Loading fresh ResNet-18 model...


...

   Splitting into train/validation sets...
   Training samples: 20000
   Validation samples: 5000

⚙️ Configuring training...

📋 Training Configuration:
   Device: CPU
   Epochs: 15
   Batch size: 8
   Training samples: 20000
   Estimated time: 60-90 minutes on CPU (larger dataset)

🚀 Starting retraining...
   This will take 60-90 minutes with the larger dataset
   You'll see progress updates every 50 steps
   Go grab some coffee! ☕



/opt/conda/lib/python3.12/site-packages/transformers/training_args.py:1626: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,4.226200,3.815608,0.248400
2,2.497800,1.921287,0.540400
3,1.278900,0.982495,0.744800
4,0.882200,0.633088,0.818200
5,0.484200,0.439687,0.868200
6,0.254100,0.354870,0.893800
7,0.209700,0.298245,0.911000
8,0.151900,0.253947,0.922600
9,0.092100,0.253820,0.923800
10,0.082700,0.256413,0.930200



✅ Retraining complete!

📊 Final Evaluation:


   Final Accuracy: 94.28%
   Final Loss: 0.2560

💾 Saving improved model...

✅ Improved model saved to: ./tesserae_google_fonts_model_v2
   This should perform MUCH better than v1!

🎉 Ready to test the improved model!


In [9]:
print("="*60)
print("TESTING IMPROVED MODEL (v2)")
print("="*60)

from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

print("\n🤖 Loading improved model v2...")

# Load your improved model
processor = AutoImageProcessor.from_pretrained("./tesserae_google_fonts_model_v2")
model = AutoModelForImageClassification.from_pretrained("./tesserae_google_fonts_model_v2")

print("✅ Model v2 loaded!")
print(f"   Can identify {len(model.config.id2label)} font families")
print(f"   Training accuracy: 94.28%")

def identify_google_font_v2(image_path):
    """Identify font using improved v2 model"""
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get top 5 predictions
    top5_prob, top5_idx = torch.topk(probabilities, 5)
    
    results = []
    for prob, idx in zip(top5_prob[0], top5_idx[0]):
        results.append({
            'font_family': model.config.id2label[idx.item()],
            'confidence': prob.item()
        })
    
    return results

# Test with your test image
print("\n🧪 Testing with test3.png...")
print("="*60)

results = identify_google_font_v2("test4.png")

print("\n🎯 MODEL v2 RESULTS:")
for i, result in enumerate(results, 1):
    print(f"  {i}. {result['font_family']:<30} {result['confidence']:>6.1%}")

print("\n📊 Comparison:")
print("   v1 (200 fonts): rochester (11.9%) ❌")
print(f"   v2 (1000 fonts): {results[0]['font_family']} ({results[0]['confidence']:.1%}) {'✅' if results[0]['confidence'] > 50 else '⚠️'}")

print("\n" + "="*60)

# Let's also check what the actual fonts in test3.png were
print("\n💡 Reminder - test4.png contains:")
print("   • Arial (or similar sans-serif)")
print("   • Courier New (monospace)")
print("   • Times New Roman (serif)")
print("   • Pacifico (script)")

print("\n" + "="*60)
print("✅ Testing complete!")

TESTING IMPROVED MODEL (v2)

🤖 Loading improved model v2...
✅ Model v2 loaded!
   Can identify 466 font families
   Training accuracy: 94.28%

🧪 Testing with test3.png...

🎯 MODEL v2 RESULTS:
  1. courierprime                    11.1%
  2. areyouserious                    5.3%
  3. cutivemono                       5.0%
  4. abyssinicasil                    3.7%
  5. amethysta                        3.0%

📊 Comparison:
   v1 (200 fonts): rochester (11.9%) ❌
   v2 (1000 fonts): courierprime (11.1%) ⚠️


💡 Reminder - test4.png contains:
   • Arial (or similar sans-serif)
   • Courier New (monospace)
   • Times New Roman (serif)
   • Pacifico (script)

✅ Testing complete!
